<a href="https://colab.research.google.com/github/athospugliese/python_codes/blob/main/Aula_3__RegressaoLogistica_EXERCICIO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<img src="https://portal.unicap.br/image/layout_set_logo?img_id=2214787&t=1657846438895" width=300, align="right">
<!-- <h1 align=left><font size = 6, style="color:rgb(200,0,0)"> optional title </font></h1> -->


# **3. Classificação: Regressão Logística**
---

Após fazer os exercícios deste laboratório responda ao **questionário correspondente da aula no Moodle**.



## Relembrando aqui...

Abaixo você pode rever o esquema geral de aplicação do modelo logístico.

In [ ]:
%%script false
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# separando variável dependente e independentes
x = df.drop('y',axis=1)
y = df['y']

# separando conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, stratify=y, random_state=1)

# selecionando o modelo
clf = LogisticRegression()

# treinando o modelo
clf.fit(X_train,y_train)

# utilizando o modelo
y_pred = clf.predict(X_test)

# verificando o resultado
print( clf.score(X_test,y_test) )

# Caso: **Consumo de combustível em veículos**

Neste Lab você vai empregar o modelo de Regressão Logística para classificar a origem de veículos a partir de dados de consumo de combustível em veículos (milhas por galão, mpg) e outras informações.

Dados (originais): https://archive.ics.uci.edu/ml/datasets/auto+mpg

## Seus imports

In [1]:
pip install ucimlrepo

In [3]:
# seu código

from ucimlrepo import fetch_ucirepo
import seaborn as sns

# fetch dataset
mpg = fetch_ucirepo(id=9)

# data (as pandas dataframes)
X = mpg.data.features
y = mpg.data.targets

# metadata
print(mpg.metadata)

# variable information
print(mpg.variables)


{'uci_id': 9, 'name': 'Auto MPG', 'repository_url': 'https://archive.ics.uci.edu/dataset/9/auto+mpg', 'data_url': 'https://archive.ics.uci.edu/static/public/9/data.csv', 'abstract': 'Revised from CMU StatLib library, data concerns city-cycle fuel consumption', 'area': 'Other', 'tasks': ['Regression'], 'characteristics': ['Multivariate'], 'num_instances': 398, 'num_features': 7, 'feature_types': ['Real', 'Categorical', 'Integer'], 'demographics': [], 'target_col': ['mpg'], 'index_col': ['car_name'], 'has_missing_values': 'yes', 'missing_values_symbol': 'NaN', 'year_of_dataset_creation': 1993, 'last_updated': 'Thu Aug 10 2023', 'dataset_doi': '10.24432/C5859H', 'creators': ['R. Quinlan'], 'intro_paper': None, 'additional_info': {'summary': 'This dataset is a slightly modified version of the dataset provided in the StatLib library.  In line with the use by Ross Quinlan (1993) in predicting the attribute "mpg", 8 of the original instances were removed because they had unknown values for th

## Load data set

In [6]:
import seaborn as sns
import pandas as pd

# Carregar o dataset mpg do seaborn
mpg = sns.load_dataset('mpg')

# Exibir as primeiras linhas do dataset para ver a estrutura
print(mpg.head())



    mpg  cylinders  displacement  horsepower  weight  acceleration  \
0  18.0          8         307.0       130.0    3504          12.0   
1  15.0          8         350.0       165.0    3693          11.5   
2  18.0          8         318.0       150.0    3436          11.0   
3  16.0          8         304.0       150.0    3433          12.0   
4  17.0          8         302.0       140.0    3449          10.5   

   model_year origin                       name  
0          70    usa  chevrolet chevelle malibu  
1          70    usa          buick skylark 320  
2          70    usa         plymouth satellite  
3          70    usa              amc rebel sst  
4          70    usa                ford torino  


# Exercício. Exclua os valores ausentes

Valores ausentes não podem ser empregados no cálculo dos coeficientes da regressão logística, e são um problema para muitos outros modelos. Existem várias formas de se tratar dados ausentes. Aqui vamos, por simplicidade, simplesmente excluir as linhas com dados ausentes.

> Qual atributo apresenta valores ausentes e quantas linhas foram excluídas dos dados?

**Dica:** Empregue `df.isna().sum()` para identificar dados ausentes e `df.dropna()` para excluir as linhas.



In [7]:
# Identificar valores ausentes
missing_values = mpg.isna().sum()
print("Valores ausentes por coluna:")
print(missing_values)

# Número de linhas antes de excluir as ausentes
num_rows_before = len(mpg)

# Excluir linhas com valores ausentes
mpg_cleaned = mpg.dropna()

# Número de linhas depois de excluir as ausentes
num_rows_after = len(mpg_cleaned)

# Número de linhas excluídas
num_rows_excluded = num_rows_before - num_rows_after

# Resultados
print(f"Número de linhas antes da exclusão: {num_rows_before}")
print(f"Número de linhas depois da exclusão: {num_rows_after}")
print(f"Número de linhas excluídas: {num_rows_excluded}")

# Mostrar as colunas que tinham valores ausentes
columns_with_missing = missing_values[missing_values > 0].index.tolist()
print(f"Colunas com valores ausentes: {columns_with_missing}")


Valores ausentes por coluna:
mpg             0
cylinders       0
displacement    0
horsepower      6
weight          0
acceleration    0
model_year      0
origin          0
name            0
dtype: int64
Número de linhas antes da exclusão: 398
Número de linhas depois da exclusão: 392
Número de linhas excluídas: 6
Colunas com valores ausentes: ['horsepower']


# Exercício. Altere os valores do campo `origin`

A Regressão Logística é um classificador binário. Altere os valores do campo `origin`:

In [ ]:
# origin estava vazio

para os valores `usa` e `non-usa` para podermos classificar a origem dos veículos como sendo dos Estados Unidos ou não.

> Quantos veículos `usa` e `non-usa` foram obtidos?

**Dica:** Empregue `df.campo.replace('old','new')`

In [8]:
mpg_cleaned['origin'] = mpg_cleaned['origin'].replace({'usa': 'usa', 'europe': 'non-usa', 'japan': 'non-usa'})

usa_count = (mpg_cleaned['origin'] == 'usa').sum()
non_usa_count = (mpg_cleaned['origin'] == 'non-usa').sum()

print("\nDataFrame modificado:")
print(mpg_cleaned.head())



DataFrame modificado:
    mpg  cylinders  displacement  horsepower  weight  acceleration  \
0  18.0          8         307.0       130.0    3504          12.0   
1  15.0          8         350.0       165.0    3693          11.5   
2  18.0          8         318.0       150.0    3436          11.0   
3  16.0          8         304.0       150.0    3433          12.0   
4  17.0          8         302.0       140.0    3449          10.5   

   model_year origin                       name  
0          70    usa  chevrolet chevelle malibu  
1          70    usa          buick skylark 320  
2          70    usa         plymouth satellite  
3          70    usa              amc rebel sst  
4          70    usa                ford torino  


<ipython-input-8-be36cb6b8b3f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mpg_cleaned['origin'] = mpg_cleaned['origin'].replace({'usa': 'usa', 'europe': 'non-usa', 'japan': 'non-usa'})


# Exercício. Classifique a `origin` dos veículos a partir dos valores de `mpg` e `cylinders`

Empregue o modelo de Regressão Logística para classificar a `origin` dos veículos a partir somente dos valores de `mpg` e `cylinders`.

Empregue $30$% para dados de teste, estratificados pelo atributo de classe e a semente de geração aleatória (`random_state=1`).

> Qual a acuracidade (*score*) obtido pelo modelo?

**Dica:** *Muito Cuidado*. Se você não empregar exatamente as configurações acima você vai criar conjuntos de treinamento e testes diferentes e não vai chegar às respostas que encontra nos testes!

In [10]:
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

mpg = sns.load_dataset('mpg')

X = mpg_cleaned[['mpg', 'cylinders']]
y = mpg_cleaned['origin'].apply(lambda x: 1 if x == 'usa' else 0)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=1)

logreg = LogisticRegression(random_state=1)
logreg.fit(X_train, y_train)


y_pred = logreg.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)

print(f"Acuracidade do modelo: {accuracy:.2f}")


Acuracidade do modelo: 0.81


# Exercício. Classifique os veículos com valores `mpg` e `cylinders` igual aos valores máximos e mínimos dos dados

Você não vai ter nenhuma surpresa aqui com o resultado :-)

> Qual o resultado da classificação?

**Dica:** Crie um dataframe exatamente com esses atributos e os valores minímo e máximos encontrados no dataframe original para sua entrada no `predict()`.


In [14]:
mpg_min, mpg_max = mpg_cleaned['mpg'].min(), mpg_cleaned['mpg'].max()
cylinders_min, cylinders_max = mpg_cleaned['cylinders'].min(), mpg_cleaned['cylinders'].max()

test_values = pd.DataFrame({
    'mpg': [mpg_min, mpg_min, mpg_max, mpg_max],
    'cylinders': [cylinders_min, cylinders_max, cylinders_min, cylinders_max]
})

# Fazer previsões com os valores extremos
predictions = logreg.predict(test_values)

# Mapeando os resultados para 'usa' e 'non-usa'
predictions_mapped = ['usa' if pred == 1 else 'non-usa' for pred in predictions]

# Exibir os valores testados e suas respectivas previsões
result_df = test_values.copy()
result_df['prediction'] = predictions_mapped

print("\nValores testados e suas respectivas previsões:")
print(result_df)



Valores testados e suas respectivas previsões:
    mpg  cylinders prediction
0   9.0          3    non-usa
1   9.0          8        usa
2  46.6          3    non-usa
3  46.6          8        usa


# Exercício. Classifique a `origin` dos veículos a partir de todos os preditores válidos

Empregue agora o modelo de Regressão Logística para classificar a `origin` dos veículos a partir de todos os preditores válidos

Empregue $30$% para dados de teste, estratificados pelo atributo de classe e a semente de geração aleatória (`random_state=1`).

> Qual a acuracidade (*score*) obtido pelo modelo?

**Dica:** *Muito Cuidado*. Se você não empregar exatamente as configurações acima você vai criar conjuntos de treinamento e testes diferentes e não vai chegar às respostas que encontra nos testes!

**Dica:** Existe um atributo que não faz sentido ser empregado como preditor e precisa ser excluído das variáveis de entrada. Qual?

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=1)

logreg = LogisticRegression(random_state=1)
logreg.fit(X_train, y_train)

y_pred = logreg.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)

print(f"Acuracidade do modelo: {accuracy:.2f}")


Acuracidade do modelo: 0.81
